# Recommender System

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

24/06/13 20:37:14 WARN Utils: Your hostname, agusrichard.local resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
24/06/13 20:37:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 20:37:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('./files/movielens_ratings.csv', inferSchema=True, header=True)
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+


In [4]:
df.describe()

DataFrame[summary: string, movieId: string, rating: string, userId: string]

In [13]:
train_set, test_set= df.randomSplit([0.8, 0.2], seed=21)

In [14]:
from pyspark.ml.recommendation import ALS

In [15]:
model = ALS(maxIter=5, regParam=0.01, userCol='userId', ratingCol='rating', itemCol='movieId')

In [16]:
model = model.fit(train_set)

In [17]:
predictions = model.transform(test_set)

In [18]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      7|   1.0|    28|  3.8796492|
|      5|   2.0|    26|  1.2893656|
|      5|   2.0|    22|-0.37337485|
|      4|   2.0|     1|  1.4325057|
|      6|   1.0|     6| 0.96962756|
|      6|   2.0|    16| 0.87772256|
|      2|   2.0|    20|-0.27982447|
|      6|   1.0|    20|  1.7459638|
|      5|   1.0|     5|  0.6788409|
|      5|   2.0|    15|  0.2576093|
|      7|   5.0|     9|   2.816122|
|      6|   1.0|     4|  1.5332997|
|      5|   1.0|     8|  1.8400079|
|      0|   1.0|    23|  1.6167369|
|      6|   3.0|    24|  1.6449468|
|      7|   1.0|    24| 0.48206958|
|      5|   1.0|    29|  1.0689286|
|      6|   2.0|    11|   2.482323|
|      1|   1.0|    14|  3.2095912|
|      4|   1.0|    14|   1.173182|
+-------+------+------+-----------+


In [19]:
from pyspark.ml.evaluation import RegressionEvaluator

In [22]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [23]:
rmse = evaluator.evaluate(predictions)
rmse

1.6905918765503232

In [25]:
single_user = test_set.filter(test_set['userId']==11).select(['movieId','userId'])

In [26]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     12|    11|
|     13|    11|
|     16|    11|
|     18|    11|
|     22|    11|
|     23|    11|
|     45|    11|
|     47|    11|
|     48|    11|
|     59|    11|
|     67|    11|
|     77|    11|
|     86|    11|
|     89|    11|
+-------+------+


In [27]:
reccomendations = model.transform(single_user)
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     22|    11|     4.02939|
|     77|    11|    3.696883|
|     47|    11|   3.4542265|
|     18|    11|   2.7679029|
|     12|    11|   2.5746477|
|      6|    11|    2.482323|
|     59|    11|   2.4171772|
|     16|    11|   2.3616102|
|     23|    11|   2.0331204|
|     13|    11|   1.5594766|
|     48|    11|   1.5316263|
|     86|    11|   1.3266013|
|     45|    11|    1.008357|
|     67|    11|   0.8548553|
|     89|    11|-0.071447566|
+-------+------+------------+
